In [1]:
import pandas as pd
import numpy as np

In [15]:
import statsmodels
import scipy 

import matplotlib.pyplot as plt
%pylab inline

import warnings
warnings.filterwarnings('ignore')

from statsmodels.stats.weightstats import *

Populating the interactive namespace from numpy and matplotlib


In [65]:
from statsmodels.sandbox.stats.multicomp import multipletests 

In [34]:
def write_answer(task, n_diff):
    with open("biology answers/" + str(task)+".txt", "w") as fout:
        fout.write(str(n_diff))

Часть 1: применение t-критерия Стьюдента

В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

для групп normal (control) и early neoplasia (treatment)

для групп early neoplasia (control) и cancer (treatment)

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.



In [3]:
data = pd.read_csv('../data/gene_high_throughput_sequencing.csv')

In [6]:
data.shape

(72, 15750)

In [7]:
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


попробовала сделать по-отдельности для 1 колонки и сравнить по стьюденту средние

In [10]:
first = data[data['Diagnosis']=='normal']['LOC643837']

In [19]:
second = data[data['Diagnosis']=='early neoplasia']['LOC643837']

In [23]:
scipy.stats.ttest_ind(first, second, equal_var = False)[1]

0.6907660157497355

код для всего дата-сета

In [33]:
data.columns[2:]

Index(['LOC643837', 'LOC100130417', 'SAMD11', 'NOC2L', 'KLHL17', 'PLEKHN1',
       'C1orf170', 'HES4', 'ISG15', 'AGRN',
       ...
       'CLIC2', 'RPS4Y1', 'ZFY', 'PRKY', 'USP9Y', 'DDX3Y', 'CD24', 'CYorf15B',
       'KDM5D', 'EIF1AY'],
      dtype='object', length=15748)

In [58]:
p_value = 0
p_values = []

for gen in data.columns[2:]:
    norm = data[data['Diagnosis']=='normal'][gen]
    early = data[data['Diagnosis']=='early neoplasia'][gen]
    
    p = scipy.stats.ttest_ind(norm, early, equal_var = False)[1]
    p_values.append(p)
    
    if p<=0.05:
        p_value+=1
print(p_value)

1575


In [36]:
write_answer('task1', p_value)

In [57]:
p_value2 = 0
p_values2 = []

for gen in data.columns[2:]:
    early = data[data['Diagnosis']=='early neoplasia'][gen]
    cancer = data[data['Diagnosis']=='cancer'][gen]
    
    p2 = scipy.stats.ttest_ind(cancer, early, equal_var = False)[1]
    p_values2.append(p2)
    if p2<=0.05:
        p_value2+=1
        
print(p_value2)

3490


In [39]:
write_answer('task2', p_value2)

#### Практическая значимость изменения

Цель исследований — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого часто используется метрика, которая называется fold change (кратность изменения). Определяется она следующим образом:

Fc(C,T)=
* T/C когда T>C 

* −C/T когда T<C


где 
C,T — средние значения экспрессии гена в control и treatment группах соответственно. По сути, fold change показывает, во сколько раз отличаются средние двух выборок.

In [103]:
# normal (control) и early neoplasia (treatment)
fold_changes = []

for gen in data.columns[2:]:
    control = data[data['Diagnosis']=='normal'][gen].mean()
    treatment = data[data['Diagnosis']=='early neoplasia'][gen].mean()
    if treatment>control:
        fold_change = treatment/control
    else:
        fold_change = -control/treatment
        
    fold_changes.append(fold_change)

In [115]:
# early neoplasia (control) и cancer (treatment)
fold_changes2 = []

for gen in data.columns[2:]:
    control = data[data['Diagnosis']=='early neoplasia'][gen].mean()
    treatment = data[data['Diagnosis']=='cancer'][gen].mean()
    if treatment>control:
        fold_change2 = treatment/control
    else:
        fold_change2 = -control/treatment
        
    fold_changes2.append(fold_change2)

Часть 2: поправка методом Холма

Для этой части задания вам понадобится модуль multitest из statsmodels.

In [40]:
import statsmodels.stats.multitest as smm

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 

1.5.

Обратите внимание, что

* применять поправку на множественную проверку нужно ко всем значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия.

* при использовании поправки на уровне значимости 0.025 меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом 0.025, а не 0.05)!

In [132]:
reject, p_corrected, a1, a2 = multipletests(p_values, 
                                            alpha = 0.05/2, 
                                            method = 'holm') 

In [133]:
n1 = 0
for i, j in zip(fold_changes, p_corrected):
    if abs(i)>1.5 and j<0.025:
        n1+=1
        print(i, j)
print('количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони'\
              'и бсолютное значение fold change > 1.5: =', n1)

1.509785482044398 0.012527423113260612
1.9748676656368698 0.0013383818900916007
количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферронии бсолютное значение fold change > 1.5: = 2


In [134]:
write_answer('task3', n1)

In [135]:
reject2, p_corrected2, a1, a2 = multipletests(p_values2, 
                                            alpha = 0.05/2, 
                                            method = 'holm') 

In [136]:
n2 = 0
for i, j in zip(fold_changes2, p_corrected2):
    if abs(i)>1.5 and j<0.025:
        n2+=1
print('количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони'\
              'и бсолютное значение fold change > 1.5: =', n2)

количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферронии бсолютное значение fold change > 1.5: = 77


In [137]:
write_answer('task4', n2)

Часть 3: поправка методом Бенджамини-Хохберга

Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от 

H 
0
​	 , когда они есть, и будут чаще отклонять 

H 
0
​	 , когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5.

In [138]:
reject_fdr_bh, p_corrected_fdr_bh, a1, a2 = multipletests(p_values, 
                                            alpha = 0.05/2, 
                                            method = 'fdr_bh')

In [139]:
n3 = 0
for i, j in zip(fold_changes, p_corrected_fdr_bh):
    if abs(i)>1.5 and j<0.025:
        n3+=1
        print(i, j)
print('количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга'\
              'и бсолютное значение fold change > 1.5: =', n3)

1.6638260655780157 0.008748910587714938
1.509785482044398 0.006264109328368201
1.7549510542769282 0.008748910587714938
1.9748676656368698 0.0013383818900916007
количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохбергаи бсолютное значение fold change > 1.5: = 4


In [140]:
write_answer('task5', n3)

In [141]:
reject_fdr_bh_2, p_corrected_fdr_bh_2, a1, a2 = multipletests(p_values2, 
                                            alpha = 0.05/2, 
                                            method = 'fdr_bh')

In [145]:
n4 = 0
for i, j in zip(fold_changes2, p_corrected_fdr_bh_2):
    if abs(i)>1.5 and j<0.025:
        n4+=1
print('количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга'\
              'и бсолютное значение fold change > 1.5: =', n4)

количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохбергаи бсолютное значение fold change > 1.5: = 524


In [146]:
write_answer('task6', n4)